In [ ]:
import pandas as pd
import numpy as np

csv_df = pd.read_csv('H28.csv',encoding="utf8") 
len(csv_df)
csv_df.info()
csv_df.head()

csv_df2 = pd.read_csv('H29.csv',encoding="utf8") 
len(csv_df2)
csv_df2.info()
csv_df2.head()

csv_df3 = pd.read_csv('H30.csv',encoding="utf8") 
len(csv_df3)
csv_df3.info()
csv_df3.head()

csv_df4 = pd.read_csv('R1.csv',encoding="utf8") 
len(csv_df4)
csv_df4.info()
csv_df4.head()

csv_df5 = pd.read_csv('R2.csv',encoding="utf8") 
len(csv_df5)
csv_df5.info()
csv_df5.head()

In [ ]:
def scaler(array, min_scale, max_scale):
    
    M = max(array)
    m = min(array)
    if M != m:
        scaled_array = (array - m*np.ones(shape = array.shape))/(M - m)
    else:
        scaled_array = array - m*np.ones(shape = array.shape)
    
    return scaled_array

def preprocessing(x):
    xl = []
    for i in range(len(x.T)):
        l = scaler(x.T[i], 0, 1)
        xl.append(l)
    
    return np.array(xl).T

def nearvalue(x, l):
    d = []
    for i in range(len(l)):
        d.append(abs(x-l[i]))
        
    return l[d.index(min(d))]

def r_2(v, w):
    
    s1 = np.sum((v-w)**2)
    s2 = np.sum((v-np.mean(v))**2)
    
    return 1-(s1/s2)

def accuracy(v,w):
    u = v-w
    a = 0
    for d in u:
        if d != 0:
            a+=1
            
    return 1-(a/len(u))

In [ ]:
import numpy as np
import time
import sklearn
ev = csv_df.drop(columns =['所在大字', '所在本番', '所在枝番', '所在小枝番', '所在地番区分',
                           '所在地番分割', '所在小字', '現況地目コード'])
ev2 = csv_df2.drop(columns =['所在大字', '所在本番', '所在枝番', '所在小枝番', '所在地番区分',
                           '所在地番分割', '所在小字', '現況地目コード'])
ev3 = csv_df3.drop(columns =['所在大字', '所在本番', '所在枝番', '所在小枝番', '所在地番区分',
                           '所在地番分割', '所在小字', '現況地目コード'])
ev4 = csv_df4.drop(columns =['所在大字', '所在本番', '所在枝番', '所在小枝番', '所在地番区分',
                           '所在地番分割', '所在小字', '現況地目コード'])
ev5 = csv_df5.drop(columns =['所在大字', '所在本番', '所在枝番', '所在小枝番', '所在地番区分',
                           '所在地番分割', '所在小字', '現況地目コード'])

x_H28 = np.array(ev.values)
y_H28 = np.array(csv_df['現況地目コード'].values)
x_H29 = np.array(ev2.values)
y_H29 = np.array(csv_df2['現況地目コード'].values)
x_H30 = np.array(ev3.values)
y_H30 = np.array(csv_df3['現況地目コード'].values)
x_R1 = np.array(ev4.values)
y_R1 = np.array(csv_df4['現況地目コード'].values)
x_R2 = np.array(ev5.values)
y_R2 = np.array(csv_df5['現況地目コード'].values)

x =np.concatenate([x_H28, x_H29])
x =np.concatenate([x, x_H30])
x =np.concatenate([x, x_R1])
x =np.concatenate([x, x_R2])
y =np.concatenate([y_H28, y_H29])
y =np.concatenate([y, y_H30])
y =np.concatenate([y, y_R1])
y =np.concatenate([y, y_R2])
y_class = list(set(y))

x = preprocessing(x)
x_H28 = x[0:len(x_H28)]
x_H29 = x[len(x_H28):len(x_H28)+len(x_H29)]
x_H30 = x[len(x_H28)+len(x_H29):len(x_H28)+len(x_H29)+len(x_H30)]
x_R1 = x[len(x_H28)+len(x_H29)+len(x_H30):len(x_H28)+len(x_H29)+len(x_H30)+len(x_R1)]
x_R2 = x[len(x_H28)+len(x_H29)+len(x_H30)+len(x_R1):]

In [ ]:
#H28を学習してH29をテスト
from sklearn.linear_model import LinearRegression
import numpy as np

lr = LinearRegression()

x_train = x_H28
y_train = y_H28

lr.fit(x_train,y_train)

print(pd.DataFrame({"Name":ev.columns,
                    "Coefficients":np.abs(lr.coef_)}).sort_values(by='Coefficients') )
print(lr.intercept_)

x_test = x_H29
y_test = y_H29

y_tr_predict = [nearvalue(i, y_class) for i in lr.predict(x_train)]
y_predict = [nearvalue(i, y_class) for i in lr.predict(x_test)]

from sklearn.metrics import mean_squared_error
import sklearn
r2_1 = r_2(y_predict, y_test)
accuracy1 = accuracy(y_predict,y_test)
print(f"決定係数{r2_1}")
print(f"正解率{accuracy1}")

#predicted = lr.predict(x2)

#print("classification report")
#print(sklearn.metrics.classification_report(y2, predicted))

print('MSE train data: ', np.sqrt(mean_squared_error(y_train, y_tr_predict))) # 学習データを用いたときの平均二乗誤差を出力
print('MSE test data: ', np.sqrt(mean_squared_error(y_test, y_predict)))         # 検証データを用いたときの平均二乗誤差を出力

In [ ]:
#H28とH29のデータを学習しH30をテスト
lr2 = LinearRegression()

x_train2 = np.concatenate([x_train,x_test])
y_train2 = np.concatenate([y_train,y_test])

lr2.fit(x_train2, y_train2)

print(pd.DataFrame({"Name":ev2.columns,
                    "Coefficients":np.abs(lr2.coef_)}).sort_values(by='Coefficients') )
print(lr2.intercept_)

x_test2 = x_H30
y_test2 = y_H30

y_tr_predict2 = [nearvalue(i, y_class) for i in lr2.predict(x_train2)]
y_predict2 = [nearvalue(i, y_class) for i in lr2.predict(x_test2)]


from sklearn.metrics import mean_squared_error
import sklearn
r2_2 = r_2(y_predict2, y_test2)
accuracy2 = accuracy(y_predict2,y_test2)
print(f"決定係数{r2_2}")
print(f"正解率{accuracy2}")


print('MSE train data: ', np.sqrt(mean_squared_error(y_train2, y_tr_predict2))) # 学習データを用いたときの平均二乗誤差を出力
print('MSE test data: ', np.sqrt(mean_squared_error(y_test2, y_predict2)))         # 検証データを用いたときの平均二乗誤差を出力

In [ ]:
#H28~H30のデータを学習しR1をテスト
lr3 = LinearRegression()

x_train3 = np.concatenate([x_train2,x_test2])
y_train3 = np.concatenate([y_train2,y_test2])

lr3.fit(x_train3, y_train3)

print(pd.DataFrame({"Name":ev3.columns,
                    "Coefficients":np.abs(lr3.coef_)}).sort_values(by='Coefficients') )
print(lr3.intercept_)

x_test3 = x_R1
y_test3 = y_R1

y_tr_predict3 = [nearvalue(i, y_class) for i in lr3.predict(x_train3)]
y_predict3 = [nearvalue(i, y_class) for i in lr3.predict(x_test3)]

from sklearn.metrics import mean_squared_error
import sklearn
r2_3 = r_2(y_predict3, y_test3)
accuracy3 = accuracy(y_predict3,y_test3)
print(f"決定係数{r2_3}")
print(f"正解率{accuracy3}")


print('MSE train data: ', np.sqrt(mean_squared_error(y_train3, y_tr_predict3))) # 学習データを用いたときの平均二乗誤差を出力
print('MSE test data: ', np.sqrt(mean_squared_error(y_test3, y_predict3)))         # 検証データを用いたときの平均二乗誤差を出力

In [ ]:
#H28~H30のデータを学習しR2をテスト
lr4 = LinearRegression()

x_train4 = np.concatenate([x_train3,x_test3])
y_train4 = np.concatenate([y_train3,y_test3])

lr4.fit(x_train4, y_train4)

print(pd.DataFrame({"Name":ev4.columns,
                    "Coefficients":np.abs(lr4.coef_)}).sort_values(by='Coefficients') )
print(lr4.intercept_)

x_test4 = x_R2
y_test4 = y_R2

y_tr_predict4 = [nearvalue(i, y_class) for i in lr4.predict(x_train4)]
y_predict4 = [nearvalue(i, y_class) for i in lr4.predict(x_test4)]

from sklearn.metrics import mean_squared_error
import sklearn
r2_4 = r_2(y_predict4, y_test4)
accuracy4 = accuracy(y_predict4,y_test4)
print(f"決定係数{r2_4}")
print(f"正解率{accuracy4}")

print('MSE train data: ', np.sqrt(mean_squared_error(y_train4, y_tr_predict4))) # 学習データを用いたときの平均二乗誤差を出力
print('MSE test data: ', np.sqrt(mean_squared_error(y_test4, y_predict4)))         # 検証データを用いたときの平均二乗誤差を出力